In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from collections import Counter
import matplotlib.pyplot as plt
from collections import OrderedDict

In [2]:
df = pd.read_json('./Data/db_export.csv', encoding='ISO-8859-1')
df.info()
df.head()
# df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 0 to 52
Data columns (total 23 columns):
1_1                      52 non-null float64
1_2                      52 non-null float64
1_3                      52 non-null float64
1_4                      52 non-null float64
1_5                      52 non-null float64
1_6                      52 non-null object
2_1                      52 non-null object
2_2                      52 non-null float64
2_3                      52 non-null float64
2_4                      52 non-null float64
3_1                      52 non-null float64
3_2                      52 non-null float64
3_3                      52 non-null float64
3_4                      52 non-null float64
3_5                      52 non-null float64
3_6                      52 non-null float64
3_7                      52 non-null float64
computerButtonClicked    52 non-null float64
currentPage              1 non-null float64
dropout                  1 non-null object
em

,1_1,1_2,1_3,1_4,1_5,1_6,2_1,2_2,2_3,2_4,...,3_4,3_5,3_6,3_7,computerButtonClicked,currentPage,dropout,email,themeChosen,timeToComplete
0,2.0,1.0,1.0,2.0,2.0,"[1, 2, 4, 5]","[1, 7]",1.0,2.0,2.0,...,4.0,3.0,2.0,1.0,0.0,NaN,NaN,tlkramer@smu.edu,3,51.131
1,1.0,1.0,1.0,1.0,1.0,"[1, 3, 4, 5, 6]",[5],2.0,2.0,2.0,...,4.0,3.0,2.0,1.0,0.0,NaN,NaN,dcsanchez@smu.edu,1,317.141
2,2.0,1.0,1.0,2.0,2.0,"[1, 2, 4, 5]",[5],1.0,2.0,2.0,...,3.0,2.0,3.0,1.0,0.0,NaN,NaN,ccjohnson@smu.edu,1,179.159
3,1.0,1.0,1.0,2.0,3.0,"[1, 2, 4]",[7],1.0,2.0,2.0,...,4.0,3.0,3.0,1.0,0.0,NaN,NaN,awoodruff@smu.edu,2,84.053
4,1.0,1.0,2.0,3.0,3.0,"[1, 2, 4, 5]","[1, 4]",1.0,2.0,2.0,...,4.0,1.0,1.0,1.0,0.0,NaN,NaN,chalquist@smu.edu,3,73.996


First, we'll need to extract the questions and their choices from the webpage

In [3]:
# open the webpage to parse for questions
with open ('./../AB-Testing-Form/templates/index.phtml') as fp:
    soup = BeautifulSoup(fp, 'html.parser')

In [4]:
survey_questions = {}  # question : [choices]

In [5]:
# for each page in the form
pages = soup.find_all('div', class_='page')
for p in range(len(pages)):
    
    if p > 0:  # ignore the first page
        questions = pages[p].find_all('div', class_='row section')
        
        for q in questions:
            # strip the number from the question
            question_title = q.find('h5').contents[1]
            
            # option tags surrounding the choices
            options = q.find('div', class_="input-field col s12 m8"
                            ).find('select').find_all('option')
            
            choices = []
            
            # keep track of those choices that aren't disabled
            for o in options:
                if 'disabled' not in str(o):
                    choices.append(o.contents[0])

            survey_questions[question_title] = choices 

In [6]:
# example of dropout
df.iloc[6]

1_1                               NaN
1_2                               NaN
1_3                               NaN
1_4                               NaN
1_5                               NaN
1_6                               NaN
2_1                               NaN
2_2                               NaN
2_3                               NaN
2_4                               NaN
3_1                               NaN
3_2                               NaN
3_3                               NaN
3_4                               NaN
3_5                               NaN
3_6                               NaN
3_7                               NaN
computerButtonClicked             NaN
currentPage                         1
dropout                          true
email                    eric@smu.edu
themeChosen                         3
timeToComplete                    NaN
Name: 6, dtype: object

In [15]:
# list of dictionaries containing responses and their occurrences
responses = []

# remove those instances that weren't completed
df_questions = df[df.dropout != 'true']

# remove non-questions from the data
df_questions = df_questions.iloc[:, :-6]  

for column in df_questions:
    col = df_questions[column]
    
    # questions with one option
    if col.dtype == "float64":
        responses.append(dict(sorted(Counter(col).items())))
    else:  # questions with multiple values
        
        # join the responses
        combined_responses = [int(inner) for outer in col 
                              for inner in outer]
        responses.append(dict(sorted(Counter(combined_responses).items())))

In [16]:
responses

[{1.0: 36, 2.0: 15, 3.0: 1},
 {1.0: 36, 2.0: 16},
 {1.0: 47, 2.0: 5},
 {1.0: 12, 2.0: 33, 3.0: 7},
 {1.0: 21, 2.0: 23, 3.0: 8},
 {1: 52, 2: 38, 3: 18, 4: 47, 5: 49, 6: 9},
 {1: 21, 3: 4, 4: 14, 5: 19, 7: 2},
 {1.0: 48, 2.0: 4},
 {1.0: 10, 2.0: 40, 3.0: 2},
 {1.0: 20, 2.0: 30, 3.0: 1, 4.0: 1},
 {1.0: 3, 2.0: 9, 3.0: 19, 4.0: 21},
 {1.0: 1, 2.0: 2, 3.0: 12, 4.0: 37},
 {1.0: 1, 2.0: 5, 3.0: 46},
 {1.0: 1, 2.0: 3, 3.0: 3, 4.0: 45},
 {1.0: 12, 2.0: 11, 3.0: 28, 4.0: 1},
 {1.0: 5, 2.0: 17, 3.0: 19, 4.0: 11},
 {1.0: 49, 2.0: 2, 3.0: 1}]

In [ ]:
for r in range(len(responses)):
    # total numbers of times options were chosen
    totalled_answers = responses[r].values()
    
    # text of questions and answers
    title = list(survey_questions.keys())[r]
    choices = list(survey_questions.values())[r]
    
    selected_choices = []
    
    # remove the choices that were never selected
    for c in range(1, len(choices)+1):
        if c in sorted(responses[r].keys()):
            print(str(c) + " is in " + str(sorted(responses[r].keys())))
            selected_choices.append(choices[c-1])
            
    # logic for building bar chart
    y_pos = np.arange(len(choices))
    plt.figure(figsize=(10,5))
    plt.bar(y_pos, totalled_answers, align='center')
    plt.xticks(y_pos, choices)
    plt.xlabel('Answer choices')
    plt.ylabel('Number of times selected')
    plt.title(title)
    
    plt.show()